In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = [[1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0],
        [0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0], 
        [0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 3, 0, 3]]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.axes.get_yaxis().set_visible(False)
ax.set_aspect(1)

def avg(a, b):
    return (a + b) / 2.0

for y, row in enumerate(data):
    for x, col in enumerate(row):
        x1 = [x, x+1]
        y1 = np.array([y, y])
        y2 = y1+1
        if col == 1:
            plt.fill_between(x1, y1, y2=y2, color='red')
            plt.text(avg(x1[0], x1[1]), avg(y1[0], y2[0]), "A", 
                                        horizontalalignment='center',
                                        verticalalignment='center')
        if col == 2:
            plt.fill_between(x1, y1, y2=y2, color='orange')
            plt.text(avg(x1[0], x1[0]+1), avg(y1[0], y2[0]), "B", 
                                        horizontalalignment='center',
                                        verticalalignment='center')
        if col == 3:
            plt.fill_between(x1, y1, y2=y2, color='yellow')
            plt.text(avg(x1[0], x1[0]+1), avg(y1[0], y2[0]), "C", 
                                        horizontalalignment='center',
                                        verticalalignment='center')

plt.ylim(3, 0)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import string

x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
y = np.array([0, 0, 1, 0, 0, 0, 1, 1, 2, 2, 0, 2, 0, 2, 4])

plt.barh(y, [1]*len(x), left=x, color = 'red', edgecolor = 'red', align='center', height=1)

plt.ylim(max(y)+0.5, min(y)-0.5)

labels = np.array(list(string.uppercase))    
plt.yticks(np.arange(y.max()+1), labels)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as md
import numpy as np
import datetime as dt
import time

n=20
duration=1000
now=time.mktime(time.localtime())
#print now
timestamps=np.linspace(now,now+duration,n)
print timestamps[9]
dates=[dt.datetime.fromtimestamp(ts) for ts in timestamps]
print dates[0]
values=np.sin((timestamps-now)/duration*2*np.pi)

plt.subplots_adjust(bottom=0.2)
plt.xticks( rotation=25 )
ax=plt.gca()
xfmt = md.DateFormatter('%Y-%m-%d %H:%M:%S')
ax.xaxis.set_major_formatter(xfmt)
plt.plot(dates,values)
plt.show()

In [ ]:
import json
import datetime as dt

# is there a more elegant way? What if linewise send per ajax-post?
f = open('../profile.log', 'r')
data_json = f.readlines()
f.close()
#print data_json[0]
data = [json.loads(line) for line in data_json]
#print data[0]['time']
#print dt.datetime.strptime(data[0]['time'], "%Y-%m-%d %H:%M:%S,%f") #2003-07-08 16:49:45,896
def make_dt(data_line):
    return dt.datetime.strptime(data_line['time'], "%Y-%m-%d %H:%M:%S,%f") #2003-07-08 16:49:45,896

#print data[0]['time']
for data_line in data:
    data_line['time'] = make_dt(data_line)
#print data[0]['time']

#print dt.strptime(data[0]['time'])
#plt.plot([data[0]['time'], data[1]['time'], data[2]['time'], data[3]['time'], data[4]['time'], data[5]['time']],
#         [0, 1, 2, 3, 0, 0])
#plt.show()
x = []
y = []
cnt = 0
for event in data:
    if event['is_start']:
        cnt += 1
    else:
        cnt -= 1
    y.append(cnt)
    x.append(event['time'])

plt.plot(x,y)
plt.show()

#print x
#print y

x = range(len(y))
y = np.array(y)

plt.barh(y, [1]*len(x), left=x, color = 'red', edgecolor = 'red', align='center', height=1)

#plt.ylim(max(y)+0.5, min(y)-0.5)

labels = np.array(list(string.uppercase))    
plt.yticks(np.arange(y.max()+1), labels)
plt.show()